In [1]:
!pip install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 16.7 MB/s eta 0:00:00a 0:00:01


In [16]:
import csv
import requests
import psycopg2
import base64
import json

# Fakturoid API credentials and parameters
FAKTUROID_API_URL = 'https://app.fakturoid.cz/api/v2/accounts/{slug}/invoices.json'  # Replace {slug} with your actual slug
FAKTUROID_USERNAME = 'jan.kadlecek+dev@marketing.bi'
FAKTUROID_API_KEY = '31f47e8632bd4b8e77f3dc7b19c7e2c945c3108b'
FAKTUROID_SLUG = 'datadrivenmarketingdev'  # Replace with your specific slug

# PostgreSQL connection parameters
PG_PARAMS = {
    "dbname": "mydatabase",
    "user": "myuser",
    "password": "mypassword",
    "host": "postgres",
    "port": "5432"
}


# Create the 'invoices' table in PostgreSQL
create_table_query = """
CREATE TABLE IF NOT EXISTS invoices (
    id SERIAL PRIMARY KEY,
    invoice_number VARCHAR(50),
    amount NUMERIC,
    customer_name VARCHAR(100)
    -- Add more columns as needed to match your data
);
"""

# Fetch invoices from Fakturoid API with slug parameter
# ... (same as before)

# Connect to PostgreSQL and create the table
with psycopg2.connect(**PG_PARAMS) as conn:
    with conn.cursor() as cur:
        cur.execute(create_table_query)

# Fetch invoices from Fakturoid API with slug parameter
url_with_slug = FAKTUROID_API_URL.format(slug=FAKTUROID_SLUG)
headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Basic ' + base64.b64encode(f"{FAKTUROID_USERNAME}:{FAKTUROID_API_KEY}".encode()).decode()
}


# TO DO - while cycle for paging, ideally to UNION to all invoices
response = requests.get(url_with_slug, headers=headers)
response.raise_for_status()
invoices = response.json()
print(response.headers)
print(json.dumps(invoices[0], indent=4))
print(len(invoices))

# Write fetched data to a CSV file
csv_file = 'invoices.csv'
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    csv_writer = csv.writer(file)
    csv_writer.writerow(invoices[0].keys())  # Assuming headers are present in the invoices data
    for invoice in invoices:
        csv_writer.writerow(invoice.values())

# Connect to PostgreSQL
with psycopg2.connect(**PG_PARAMS) as conn:
    with conn.cursor() as cur:
        # Here you can load the CSV data into the database
        with open(csv_file, 'r', encoding='utf-8') as f:
            next(f)  # Skip header row
            cur.copy_from(f, 'invoices.csv', sep=',')
    conn.commit()

{'access-control-allow-origin': '*', 'cache-control': 'max-age=0, private, must-revalidate', 'content-encoding': 'gzip', 'content-type': 'application/json; charset=utf-8', 'etag': 'W/"97b81c46ef729b8d2de2647a1874fa31"', 'last-modified': 'Sun, 12 Feb 2023 23:38:43 GMT', 'referrer-policy': 'strict-origin-when-cross-origin', 'set-cookie': 'application_theme=eyJfcmFpbHMiOnsibWVzc2FnZSI6IklteHBaMmgwSWc9PSIsImV4cCI6IjIwMjQtMTItMDVUMTI6Mjg6MzQuODA5WiIsInB1ciI6ImNvb2tpZS5hcHBsaWNhdGlvbl90aGVtZSJ9fQ%3D%3D--251eacc274134477e4f6442cd5c0d26f7f56bc2b; path=/; expires=Thu, 05 Dec 2024 12:28:34 GMT; HttpOnly; SameSite=Lax; secure', 'strict-transport-security': 'max-age=63072000; includeSubDomains; preload', 'vary': 'Accept-Encoding', 'x-content-type-options': 'nosniff', 'x-download-options': 'noopen', 'x-frame-options': 'SAMEORIGIN', 'x-permitted-cross-domain-policies': 'none', 'x-request-id': '653ded79-0895-442f-aa17-842fb6d08037', 'x-runtime': '0.032975', 'x-xss-protection': '0', 'date': 'Tue, 05 D

BadCopyFileFormat: extra data after last expected column
CONTEXT:  COPY invoices, line 1: "23125376,,False,False,0021,397206,21,Testovací účet,Testovací ,,Test,10000,CZ,,,,JVJ agency s.r...."
